In [1]:
!pip install --quiet -U gymnasium[classic-control]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 11.0 MB/s eta 0:00:00


In [64]:
import gymnasium as gym
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

In [65]:
gamma = 0.99 # Discount factor for rewards

class Policy(nn.Module):
  def __init__(self, in_dim, out_dim):
    super(Policy, self).__init__()
    layers = [
        nn.Linear(in_dim, 64),
        nn.ReLU(),
        nn.Linear(64, out_dim),
    ]
    self.model = nn.Sequential(*layers) # Model is a Multi-Layer Perceptron of size in_dim, 64, out_dim with the Rectified Linear Unit non-linearity
    self.onpolicy_reset()
    self.train() # set training mode

  def onpolicy_reset(self):
    self.log_probs = []
    self.rewards = []

  def forward(self, x):
    return self.model(x)

  def act(self, state):
    x = torch.from_numpy(state.astype(np.float32)) # Transform state to PyTorch Tensor filled with 32 bit floats
    pdparam = self.forward(x) # Forward pass (pass state through model) to generate action probabilities
    pd = Categorical(logits=pdparam) # Create Probability Distribution
    action = pd.sample() # Sample probability distribution [policy(a|s)] to get action
    log_prob = pd.log_prob(action) # log_prob of policy(a|s)
    self.log_probs.append(log_prob) # store for training
    return action.item() # .item() unwraps action's metadata

In [66]:
def train(policy, optimizer):
  # Inner gradient-ascent loop of REINFORCE algorithm (https://people.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)
  T = len(policy.rewards) # Number of timesteps
  rets = np.empty(T, dtype=np.float32) # The returns
  future_ret = 0.0
  # Efficient computation of the returns
  for t in reversed(range(T)):
    future_ret = policy.rewards[t] + gamma * future_ret
    rets[t] = future_ret
  rets = torch.tensor(rets)
  log_probs = torch.stack(policy.log_probs)
  loss = -log_probs * rets # Gradient term -- negative for maximizing, pytorch's optimizer minimizes loss by default
  loss = torch.sum(loss)
  optimizer.zero_grad() # Zero out the gradients from previous training step -- crucial before calling .backward()
  loss.backward() # Backpropagate, compute gradients
  optimizer.step() # Gradient-ascent, update the weights
  return loss

In [67]:
import matplotlib.pyplot as plt

def main():
  env = gym.make('CartPole-v1', render_mode = "rgb_array")
  in_dim = env.observation_space.shape[0] # 4 observations of state per time-step: cart position, cart velocity, pole angle, pole angular velocity
  out_dim = env.action_space.n # 2 possible actions -- move cart left or right
  policy = Policy(in_dim, out_dim)
  optimizer = optim.Adam(policy.parameters(), lr=0.01)
  for episode in range(300):
    state, _ = env.reset()
    for t in range(500): # CartPole max timestep is 500
      action = policy.act(state)
      state, reward, done, _, _ = env.step(action)
      policy.rewards.append(reward)
      #env.render() # This does not work in google colab
      # [janky Colab workaround] Render the current frame and display it
      #frame = env.render()  # Returns the current frame as an ndarray
      #plt.imshow(frame)  # Display using matplotlib
      #plt.axis('off')  # Turn off axes
      #plt.show()
      if done:
        break
    loss = train(policy, optimizer) # train per episode
    total_reward = sum(policy.rewards)
    solved = total_reward > 195.0
    policy.onpolicy_reset() # REINFORCE is an on-policy algorithm, meaning we clear memory after training
    print(f'Episode {episode+1}, loss: {loss}, total_reward: {total_reward}, solved: {solved}')

In [68]:
main()

Episode 1, loss: 260.1946105957031, total_reward: 28.0, solved: False
Episode 2, loss: 104.17900085449219, total_reward: 17.0, solved: False
Episode 3, loss: 55.39462661743164, total_reward: 12.0, solved: False
Episode 4, loss: 33.61216354370117, total_reward: 10.0, solved: False
Episode 5, loss: 32.234779357910156, total_reward: 10.0, solved: False
Episode 6, loss: 187.8597412109375, total_reward: 23.0, solved: False
Episode 7, loss: 46.2695198059082, total_reward: 12.0, solved: False
Episode 8, loss: 45.58971405029297, total_reward: 12.0, solved: False
Episode 9, loss: 54.007850646972656, total_reward: 13.0, solved: False
Episode 10, loss: 141.78094482421875, total_reward: 21.0, solved: False
Episode 11, loss: 67.16352081298828, total_reward: 13.0, solved: False
Episode 12, loss: 57.35215377807617, total_reward: 13.0, solved: False
Episode 13, loss: 235.0272216796875, total_reward: 26.0, solved: False
Episode 14, loss: 28.08201789855957, total_reward: 10.0, solved: False
Episode 15, 